<a href="https://colab.research.google.com/github/Manisha-Karim/Movie-Recommendation-System/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095456 sha256=5b06c17d7da808e5f4fec24409a2e14511d0db33219fbac2caeb1e5572082381
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold


In [3]:
links_small = pd.read_csv('/content/drive/MyDrive/Datasets/movie dataset/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [4]:
links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [5]:
md = pd.read_csv("/content/drive/MyDrive/Datasets/movie dataset/movies_metadata.csv")

<ipython-input-5-6adac4dbaf41>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv("/content/drive/MyDrive/Datasets/movie dataset/movies_metadata.csv")


In [6]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# Content Based Recommendation


In [8]:
md = md.drop([19730, 29503, 35587]) #drop due to error in id
md['id'] = md['id'].astype('int')

In [9]:
df = md[md['id'].isin(links_small)]

In [10]:
df.shape

(9099, 24)

In [11]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
df['overview'][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [13]:
import re

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
snow_stemmer = SnowballStemmer(language='english')
stop = stopwords.words('english')

In [16]:
def review_to_words(description):
    # 1. Make a space
    letters_only = re.sub("[^a-zA-Z]",   " ",  description)
    # 2. lower letters
    words = letters_only.lower().split()
    # 3. Stopwords
    meaningful_words = [w for w in words if not w in stop]
    # 4. lemmitization
    stemmatize_words = [snow_stemmer.stem(w) for w in meaningful_words]
    # 5. space join words
    return( ' '.join(stemmatize_words))

In [17]:
df['overview'] = df['overview'] + df['tagline']
df['overview'] = df['overview'].fillna('')
df['overview'] = df['overview'].apply(review_to_words)

<ipython-input-17-5b110275bbb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'] = df['overview'] + df['tagline']
<ipython-input-17-5b110275bbb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'] = df['overview'].fillna('')
<ipython-input-17-5b110275bbb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [18]:
df['overview']

0                                                         
1        sibl judi peter discov enchant board game open...
2        famili wed reignit ancient feud next door neig...
3        cheat mistreat step women hold breath wait elu...
4        georg bank recov daughter wed receiv news preg...
                               ...                        
40224    mind behind evangelion come hit larger life ma...
40503    band storm europ conquer america groundbreak w...
44821    molli hale sad father disappear get unknow use...
44826                                                     
45265                                                     
Name: overview, Length: 9099, dtype: object

In [19]:
tf = TfidfVectorizer(analyzer='word',stop_words='english', max_df=0.8)
tfidf_matrix = tf.fit_transform(df['overview'])

In [20]:
tfidf_matrix.shape

(9099, 18386)

In [21]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
cosine_sim[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [23]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Celebi - Voice of the Forest           9097
Force Majeure                                         9098
Length: 9099, dtype: int64

In [24]:
def recommendation(title):
  #similar movies based on cosine similarity
  index = indices[title]
  similarity = list(enumerate(cosine_sim[index]))
  similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
  similarity = similarity[1:31]
  movie_index = [i[0] for i in similarity]
  movies = df.iloc[movie_index][['title', 'vote_count', 'vote_average']]

  #highest rated amongst similar movies using IMDB's weighted rating
  vote_count = movies['vote_count'].astype('int')
  vote_average = movies['vote_average'].astype('int')

  C = vote_average.mean()
  m = vote_count.quantile(0.60)

  qualified = movies[(movies['vote_count'] >= m)]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')

  qualified['wr'] =  qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
  qualified = qualified.sort_values('wr', ascending=False).head(10)
  return qualified

In [25]:
recommendation('Iron Man 2').head(10)

<ipython-input-24-8cdc6f999bec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
<ipython-input-24-8cdc6f999bec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
<ipython-input-24-8cdc6f999bec>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,title,vote_count,vote_average,wr
8754,Mad Max: Fury Road,9629,7,6.953666
6928,Iron Man,8951,7,6.950294
8758,Avengers: Age of Ultron,6908,7,6.936332
8666,Fury,4028,7,6.894598
8285,Iron Man 3,8951,6,5.989536
8312,Man of Steel,6462,6,5.985721
8824,Insurgent,3921,6,5.977256
9067,Now You See Me 2,3362,6,5.973846
7639,Megamind,1968,6,5.958222
4268,Saturday Night Fever,383,6,5.869766


In [26]:
recommendation('Mean Girls').head(10)

<ipython-input-24-8cdc6f999bec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
<ipython-input-24-8cdc6f999bec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
<ipython-input-24-8cdc6f999bec>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,title,vote_count,vote_average,wr
3372,Blow,1352,7,6.804837
1083,Cape Fear,692,7,6.671164
994,Manhattan,600,7,6.636454
7742,Just Go with It,1588,6,5.975658
6320,District B13,572,6,5.946341
4921,The Last Boy Scout,502,6,5.941486
7118,Wild Child,421,6,5.934642
608,The Craft,388,6,5.931373
3244,Save the Last Dance,359,6,5.928217
3776,Bandits,307,6,5.921765


# Collaborative Recommendation System


In [27]:
ratings =  pd.read_csv("/content/drive/MyDrive/Datasets/movie dataset/ratings_small.csv")

In [28]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7f0fffc74200>

In [29]:
from surprise.model_selection import cross_validate
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.90654521, 0.89551453, 0.89446524, 0.89030435, 0.89635311]),
 'test_mae': array([0.69666725, 0.68938159, 0.68945993, 0.68735361, 0.69016122]),
 'fit_time': (2.838578701019287,
  4.212174415588379,
  3.857609510421753,
  1.6328370571136475,
  1.644648790359497),
 'test_time': (0.2563488483428955,
  0.9515640735626221,
  0.3695516586303711,
  0.1435396671295166,
  0.15069937705993652)}

# Hybrid Recommendation System


In [30]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan


In [31]:
id_map = pd.read_csv('/content/drive/MyDrive/Datasets/movie dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(df[['title', 'id']], on='id').set_index('title')

In [32]:
#id_map = id_map.set_index('tmdbId')
indices_map = id_map.set_index('id')

In [33]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [34]:
hybrid(1, 'The Dark Knight')

,title,vote_count,vote_average,id,est
6144,Batman Begins,7511.0,7.5,272,3.284588
7931,The Dark Knight Rises,9263.0,7.6,49026,3.071187
2696,JFK,513.0,7.5,820,3.026598
8165,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,123025,3.007012
8227,"Batman: The Dark Knight Returns, Part 2",426.0,7.9,142061,2.882828
2579,Batman: Mask of the Phantasm,218.0,7.4,14919,2.874688
8371,Justice League: Crisis on Two Earths,152.0,7.1,30061,2.822536
7344,Law Abiding Citizen,1522.0,7.2,22803,2.766795
7274,Gangster's Paradise: Jerusalema,16.0,6.8,22600,2.735111
7242,The File on Thelma Jordon,8.0,6.3,35404,2.735111
